In [1]:
import pandas as pd
import datetime as datetime


In [ ]:
## Q2 load data 

timesdata           = pd.read_csv('GPOpeningTimes.csv',delimiter = '¬',encoding='raw_unicode_escape')
practicedata        = pd.read_csv('GPPractices.csv',delimiter = '¬',encoding='raw_unicode_escape')
transparencydata     = pd.read_csv('TransparencyIndicatorsGPPerformance.csv',delimiter = '¬',encoding='raw_unicode_escape')

In [ ]:
"""
Data exploration

CSV variables 

GP Opening time 
OrganisationId
WeekDay - NaN if additional opendate (bank hol etc)
Times
IsOpen - False when is a non weekday
OpeningTimeType
AdditionalOpeningDate

GP Practices
OrganisationID- unique 
OrganisationCode - check if unique or not 
OrganisationType - GP surgery
SubType - mostly unknown
OrganisationName
OrganisationStatus -visible/not visible
IsPimsManaged - Binary
Address1
Address2
Address3
City
County
Postcode
Latitude
Longitude

Transparency indicator performance 
OrganisationID
OrganisationCode
OrganisationName
MetricName
Value
Text
"""

# get list of column names for each dataframe 
print(practicedata.columns)
print(transparencydata.columns)


# Explore which columns have NAs to better understand the data 
times_na                = timesdata.isnull().any()
times_na_columns        = list(times_na[times_na].index)

practice_na             = practicedata.isnull().any()
practice_na_columns     = list(practice_na[practice_na].index)

transparency_na         = transparencydata.isnull().any()
transparency_na_columns = list(transparency_na[transparency_na].index)


print(times_na_columns)
print('~~~~~~~')
print(practice_na_columns)
print('~~~~~~~')
print(transparency_na_columns)



In [ ]:
""" Q3 Can you please get us a list of addresses suitable for sending a letter
of all practises whose antibiotic prescribing rate is above 0.5.
We would like this data to be exported as a CSV with the following headers 'Organisation
Name', 'Address1', 'Address2', 'Address3', 'City', 'County', 'Postcode', 'Value of antibiotic
prescribing rate'
"""

practice_columns = ['OrganisationID','OrganisationName','Address1',
       'Address2', 'Address3', 'City', 'County', 'Postcode']

transparancy_columns = ['OrganisationID','MetricName',
       'Value']

merged_inner = pd.merge( left = practicedata[practice_columns], right = transparencydata[transparancy_columns], left_on='OrganisationID', right_on ='OrganisationID')


antibiotic_inner = merged_inner[merged_inner['MetricName'] == 'Antibiotic Prescribing']
antibiotic_inner['Value'] = antibiotic_inner['Value'].astype(float)
# antibiotic_inner = antibiotic_inner.loc[antibiotic_inner['Value'] >0.5,]
antibiotic_inner = antibiotic_inner[antibiotic_inner['Value'] >0.5]

In [ ]:
## formatting output 
renamed_data =antibiotic_inner.rename({'Value':'Value of antibiotic prescribing rate'})
renamed_data =renamed_data.drop('OrganisationID', 1)
renamed_data.iloc[:,1:]
renamed_data.to_csv('antibiotic_prescribing_rate.csv',index=False)

In [6]:
"""
Q4
We do sometimes congratulate surgeries on good behaviour, opening times are a key metric
here, as we know how important it is for surgeries to be open so people can get an
appointment. Could we get a sorted list of organisation ids of surgeries by how many minutes
their reception is open per week?

Example structure
08:00-20:30

"""

"""
steps: 
1) select only reception column
2) select only when "IsOpen" is true 
3) Check "false" columns, do they have values?
4) check values for "Times"
5) create new column containing hours worked 

"""

unique_opening_types= set(timesdata['OpeningTimeType'].values)
#{'Additional', 'Reception', 'Surgery'}
unique_opening_types= set(timesdata['IsOpen'].values)

reception_week_hours = timesdata[(timesdata['OpeningTimeType'] == 'Reception' ) & (timesdata['IsOpen'] == True)]

# check format of to check the "Times" data
unique_opening_times= set(timesdata['Times'].values)


In [ ]:
def time_open_calculator(time_string):
    """
    A function to conver the "hh:mm-hh:mm" time format to a length in minutes

    input:str
    output:int
    
    """
    start_time, end_time = str(time_string).split('-')

    format = '%H:%M'
    startDateTime = datetime.datetime.strptime(start_time, format)
    endDateTime = datetime.datetime.strptime(end_time, format)

    diff = endDateTime - startDateTime
    minutes = diff.seconds//60
    return(int(minutes))


# Checking the 'Times' types are string 
reception_week_hours['Times'] =reception_week_hours['Times'].astype(str)
type_set = set([(type(value)) for value in reception_week_hours['Times']])
print(f'Types of data in Times column {str(type_set)}')

reception_week_hours['MinsOpen'] = reception_week_hours.apply(lambda x: time_open_calculator(x['Times']), axis =1)

grouped_minsopen = reception_week_hours.groupby('OrganisationId')['MinsOpen'].sum()


# We can display the order either ascending or decending 
grouped_minsopen = grouped_minsopen.to_frame()
sorted_minsopen = grouped_minsopen.sort_values('MinsOpen',ascending=True)

sorted_minsopen.to_csv('practice_by_minutes_open.csv',index=False)

In [ ]:
"""" Q5 Uploaded csv of Pratice Id/ to S3 bucket 

Step 0) Create AWS account and get secret key from IAM 

Step 1)In linux style terminal(not python consle) setup your aws keys using aws cli (must be installed also)


- aws configure
- aws_access_key_id = YOUR_ACCESS_KEY
- aws_secret_access_key = YOUR_SECRET_KEY

Step 2) Use boto library within python console/IDE to upload csv to S3 bucket 

"""
"""
# would also have to install boto3 to virtual env 
import boto3

s3 = boto3.resource('s3')

# Bucket is whatever you S3 bucketname is 
BUCKET = "NHS_data"
BUCKET_PATH = "dump/file"

s3.Bucket(BUCKET).upload_file('practice_by_minutes_open.csv', BUCKET_PATH)
"""
